In [1]:
import utilities as u

import torch
from torch_geometric.data import Data, InMemoryDataset, DataLoader
import torch.nn as nn

import pickle
import random

torch.manual_seed(42)

In [2]:
# EXPNAME = 'test_adding_edge_features'
EXPNAME = 'bug_fix'
binary = True
only_top = True

# Tensorboard Plotting 

In [3]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(f'runs/{EXPNAME}')

# Define Dataset

In [4]:
from tqdm import tqdm

class TopLevelProofDataset(InMemoryDataset):
    def __init__(self, root='', transform=None, pre_transform=None):
        super(TopLevelProofDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return [f'../bug_fix.dataset']
    
    def download(self):
        pass
    
    def process(self):
        global data
        data_list = []
        all_features = set()
        trees = []
        
        for thm, y in tqdm(data):
            thm = u.process_theorem(thm)
            tree, distinct_features = u.thm_to_tree(thm)
            all_features = all_features | distinct_features
            trees.append((tree, y))
        
        normalized_features = {k: [random.random() for i in range(128)] for k in list(all_features)}
            
        for tree, y in tqdm(trees):
            merged_tree = u.merge_subexpressions(tree)
            x, (edge_index_up, edge_index_down), (edge_features_up, edge_features_down) = u.graph_to_data(tree, 
                                                                                                           normalized_features)
            data = Data(x=x, 
                        y=y, 
                        edge_index=torch.cat((edge_index_up, edge_index_down), dim=1),
                        edge_attr=torch.cat((edge_features_up, edge_features_down)),
                       )
            data_list.append(data)
            
        
#         for thm, y in tqdm(data):
#             thm = u.process_theorem(thm)
#             tree, distinct_features = u.thm_to_tree(thm)
#             normalized_features = {k: [random.random() for i in range(16)] for k in list(distinct_features)}
#             tree = u.merge_subexpressions(tree)
            
# #             x, edge_index = u.graph_to_data(tree, list(distinct_features))
#             x, edge_index = u.graph_to_data(tree, normalized_features)
            
#             data = Data(x=x, edge_index=edge_index, y=y)
#             data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# SAGEConv Layer

In [5]:
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops

class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='mean') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

# GNN definition

In [46]:
embed_dim = 128
from torch_geometric.nn import TopKPooling, GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = SAGEConv(dataset.num_features, embed_dim)
#         self.conv1 = GCNConv(embed_dim, 128)
        self.embedding = torch.nn.Embedding(num_embeddings=len(distinct_features)+1, embedding_dim=embed_dim)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 11)
        self.lin4 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()  
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch    

        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
#         x = x1

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.lin4(x))

#         x = F.log_softmax(self.lin3(x), dim=1).squeeze(1)


        return x

# Model 2 (Subgraph Pooling Paper)

In [30]:
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops

class PaliwalMP(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(PaliwalMP, self).__init__(aggr='mean', flow='target_to_source') #  "Mean" aggregation.
        
        # MLP for Parents and Children, step 2 of Paliwal MP
        self.MLP_edge = BuildingBlock(3*128, 128)
        self.MLP_edge_hat = BuildingBlock(3*128, 128)
        
        # MLP to pass aggregated message through, step 3 of Paliwal MP
        self.MLP_aggr = BuildingBlock(3*128, 128)

        
    def forward(self, x, edge_index_parents, edge_index_children, edge_attr_parents, edge_attr_children):
        # x has shape [N, in_channels]
        # edge_index_x has shape [2, E/2]
        out_parents = self.propagate(edge_index_parents, 
                                     x=x, 
                                     edge_attr=edge_attr_parents, 
                                     direction='up', 
                                     size=None)
        out_children = self.propagate(edge_index_children, 
                                      x=x, 
                                      edge_attr=edge_attr_children, 
                                      direction='down', 
                                      size=None)
        
        out = torch.cat([x, out_parents, out_children], dim=1)
        out = self.MLP_aggr(out) + x
        
        
        return out

    
    def message(self, x_i, x_j, edge_attr, direction):

        s_ij = torch.cat([x_i, x_j, edge_attr], dim=1)
        if direction == 'up':
            s_ij = self.MLP_edge(s_ij)
        elif direction == 'down':
            s_ij = self.MLP_edge_hat(s_ij)
        
        return s_ij

    
    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]
        return aggr_out


In [76]:
from torch_geometric.nn import global_max_pool as gmp

embed_dim = 128

# TODO: Apply dropout

class BuildingBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, dim=0):
        super(BuildingBlock, self).__init__()
        self.lin1 = Linear(in_channels, 256, dim)
        self.hidden = Linear(256, 128)
        self.lin2 = Linear(128, 128)
        
    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.hidden(x))
        x = F.dropout(x, 0.5)
        x = F.relu(self.lin2(x))
        
        return x
    

class PaliwalNet(torch.nn.Module):
    def __init__(self, t):
        super(PaliwalNet, self).__init__()
        self.MLP_V = BuildingBlock(embed_dim, 128)
        self.MLP_E = BuildingBlock(1, 128)
        
        self.message_passing_steps = nn.ModuleList()
        for i in range(t):
            self.message_passing_steps.append(PaliwalMP(embed_dim, embed_dim))
            
        self.conv1 = nn.Conv1d(128, 512, (1,1))
        self.conv2 = nn.Conv1d(512, 1024, (1,1))
        
        # FCNN for final prediction
        self.lin1 = Linear(1024, 512)
        self.lin2 = Linear(512, 512)
        self.lin3 = Linear(512, 256)
        self.lin4 = Linear(256, 256)
        self.lin5 = Linear(256, 128)
        self.lin6 = Linear(128, 1)

  
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        edge_index_u, edge_index_d = torch.split(edge_index, int(edge_index.shape[1]/2), dim=1)
        edge_attr_u, edge_attr_d = torch.split(edge_attr, int(edge_attr.shape[0]/2))
        
        # Embed node and edge features into high dimensional space
        x = self.MLP_V(x)
        edge_attr_u = self.MLP_E(edge_attr_u.float())
        edge_attr_d = self.MLP_E(edge_attr_d.float())
        
        for message_passing_step in self.message_passing_steps:
            x = message_passing_step(x, edge_index_u, edge_index_d, edge_attr_u, edge_attr_d)
        
        x = x.unsqueeze(-1).unsqueeze(-1)
        x = self.conv1(x)
        x = self.conv2(x)
        
        # Final prediction network
        g = gmp(x, batch)
        g = g.squeeze(-1).squeeze(-1)
        g = F.relu(self.lin1(g))
        g = F.relu(self.lin2(g))
        g = F.relu(self.lin3(g))
        g = F.relu(self.lin4(g))
        g = F.relu(self.lin5(g))
        g = F.relu(self.lin6(g))
        
        return x

In [77]:
pnet = PaliwalNet(t=1)
net = Net()
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False, num_workers=1)

for i, data in enumerate(train_loader):
    pnet(data)
    break
    
for i, data in enumerate(train_loader):
    net(data)
    break

torch.Size([439, 128, 1, 1])
torch.Size([439, 1024, 1, 1])
torch.Size([16, 1]) tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)
hi torch.Size([439, 128])
torch.Size([16, 256])


# Data inspections

In [9]:
new_data = True

if new_data == True:
    data = u.make_data(binary=binary, only_top=only_top)
    with open(EXPNAME, 'wb') as outfile:
        pickle.dump(data, outfile)
else:
    with open(EXPNAME, 'rb') as infile:
        data = pickle.load(infile)
        
        
# data = data[0:2]

100%|██████████| 10/10 [00:07<00:00,  1.32it/s]


In [10]:
distinct_features = set()

for idx, (thm, _) in enumerate(data):
    if idx % 1000 == 0:
        print(f'{idx} / {len(data)}')
    thm = u.process_theorem(thm)
    thm_tree, features = u.thm_to_tree(thm)
    distinct_features = distinct_features.union(features)
len(distinct_features)

0 / 1427
1000 / 1427


350

In [11]:
distinct_features = set(i for i in range(len(distinct_features)))

In [12]:
test_thm = '(fun (a A B) (a A (a A B)))'
print(test_thm)
thm = u.process_theorem(test_thm)
print(thm)
thm_tree, _ = u.thm_to_tree(thm)
print(len(thm_tree))
print(thm_tree.subtrees[0].parents[0])
thm_tree = u.merge_subexpressions(thm_tree)
x = u.graph_to_data(thm_tree)
print(x)

#print([t.root for t in thm_tree.subtrees[0].subtrees])


print(thm_tree.root)
print([t.root for t in thm_tree.subtrees])
t_0, t_1 = thm_tree.subtrees
print([t.root for t in t_0.subtrees])
print([t.root for t in t_1.subtrees])
print(t_1.subtrees[0].subtree_str)
print(len(thm_tree))

(fun (a A B) (a A (a A B)))
['(', 'fun', '(', 'a', 'A', 'B', ')', '(', 'a', 'A', '(', 'a', 'A', 'B', ')', ')', ')']
9
Tree(root=(fun, index=None), parents=None, size=9)
5
(['fun', 'a', 'A', 'B', 'a'], (tensor([[1, 1, 3, 3, 4, 2],
        [0, 2, 1, 2, 1, 0]]), tensor([[0, 0, 1, 1, 2, 2],
        [2, 1, 4, 3, 1, 3]])), (tensor([[0],
        [0],
        [0],
        [0],
        [1],
        [1]]), tensor([[1],
        [0],
        [1],
        [0],
        [1],
        [0]])))
(fun, index=0)
[(a, index=1), (a, index=2)]
[(A, index=3), (B, index=4)]
[(A, index=3), (a, index=1)]
A
5


In [13]:
counter = dict()
for _, y in data:
    if y in counter:
        counter[y] += 1
    else:
        counter[y] = 1
counter = list(counter.items())
counter.sort(key=lambda x: x[0], reverse=False)
percentages = [(x, y/len(data)*100) for x,y in counter]
percentages

[(0, 45.05956552207428), (1, 54.94043447792571)]

# Create Dataset

In [14]:
from math import floor

dataset = TopLevelProofDataset()
dataset.shuffle()

train_dataset = dataset[:floor(len(dataset)/2)]
valid_dataset = dataset[floor(len(dataset)/2) : 3*floor(len(dataset)/2)]
test_dataset = dataset[3*floor(len(dataset)/2):]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=8)
print(train_dataset)

  3%|▎         | 39/1427 [00:00<00:03, 384.38it/s]

Processing...


  5%|▌         | 73/1427 [00:00<00:04, 282.32it/s]

25
15
27
26
31
31
24
31
45
25
34
22
23
17
42
21
44
46
21
49
21
38
36
36
39
38
37
39
38
37
37
38
37
36
36
36
39
38
29
28
30
32
40
46
37
16
40
20
20
41
15
17
40
34
28
21
18
19
31
31
21
19
37
19
36
36
19
39
19
37
37
41
54
54
40
41
13
62
47


 11%|█         | 151/1427 [00:00<00:04, 313.69it/s]

...

 98%|█████████▊| 1401/1427 [00:08<00:00, 166.82it/s]

81
36
64
31
25
38
26
30
34
73
64
78
84
31
92
49
55
34
38
35
46
62
59
50
49
55
34
78
21
45
22
81
77
59
69
66
27
73
67
61
46
47
110
48
50
110
50
51
110
71


100%|██████████| 1427/1427 [00:09<00:00, 157.69it/s]

286
111
139
51
81
88
98
117
144
141
34
159


Done!
TopLevelProofDataset(713)


# Train

In [54]:
batch_size = 1
num_epochs = 100

def train():
    global epoch
    model.train()
    
    loss_all = 0
    correct = 0
    for i, data in enumerate(train_loader):
        x = data.x.squeeze(1)

        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = torch.unsqueeze(data.y.to(device), 1).float()
#         torch.unsqueeze(label, 1)
#         print(data.shape, output.shape, label.shape)
#         print(label, output)
#         loss = F.nll_loss(output, label)
        loss = F.mse_loss(output, label)
    
        if torch.isnan(loss):
            print(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct_this_time = pred.eq(label.data.view_as(pred)).sum()
        correct += correct_this_time
#         if correct_this_time == 0:
#             print(label, output)
        

        optimizer.step()
#         print(correct.item())
    
    writer.add_scalar('training loss',
                     loss_all / len(train_dataset),
                     epoch)
    
    writer.add_scalar('training accuracy',
                     correct.item() / len(train_dataset),
                     epoch)
    #print(correct.item())
    
    return loss_all / len(train_dataset), correct.item() / len(train_dataset)


def test():
    global epoch
    model.eval()
    
    loss_all = 0
    correct = 0
    for i, data in enumerate(valid_loader):
        x = data.x.squeeze(1)

        data = data.to(device)
        output = model(data)
        label = torch.unsqueeze(data.y.to(device), 1)
        
        loss = F.mse_loss(output, label)
        loss_all += data.num_graphs * loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(label.data.view_as(pred)).sum()
    
    writer.add_scalar('validation loss',
                     loss_all / len(valid_dataset),
                     epoch)
    
    writer.add_scalar('validation accuracy',
                     correct.item() / len(valid_dataset),
                     epoch)
    
    return loss_all / len(valid_dataset), correct.item() / len(valid_dataset)
    
    
# device = torch.device('cuda:2')
device = torch.device('cpu')

model = Net().to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.00005, momentum=0.8)
crit = torch.nn.CrossEntropyLoss()
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

assert 1 == 2

epoch = 0
valid_loss, valid_acc = test()
for epoch in tqdm(range(num_epochs)):
    epoch_loss, epoch_acc = train()
    #print(f'train - loss: {epoch_loss}, acc: {epoch_acc}')
    
    if epoch % 5 == 4:
        valid_loss, valid_acc = test()
#         print(f'valid - loss: {valid_loss}, acc: {valid_acc}')
        
    #print()
        
    

AssertionError: 

In [60]:
def save_model(model, PATH):
    torch.save(model.state_dict(), PATH)
    
def load_model(model_type, PATH):
    model = model_type()
    model.load_state_dict(torch.load(PATH))
    return model

In [61]:
m = load_model(Net, 'bug_fix.pth')
m

Net(
  (conv1): SAGEConv(
    (lin): Linear(in_features=128, out_features=128, bias=True)
    (act): ReLU()
    (update_lin): Linear(in_features=256, out_features=128, bias=False)
    (update_act): ReLU()
  )
  (embedding): Embedding(351, 128)
  (pool1): TopKPooling(128, ratio=0.8, multiplier=1)
  (conv2): SAGEConv(
    (lin): Linear(in_features=128, out_features=128, bias=True)
    (act): ReLU()
    (update_lin): Linear(in_features=256, out_features=128, bias=False)
    (update_act): ReLU()
  )
  (pool2): TopKPooling(128, ratio=0.8, multiplier=1)
  (conv3): SAGEConv(
    (lin): Linear(in_features=128, out_features=128, bias=True)
    (act): ReLU()
    (update_lin): Linear(in_features=256, out_features=128, bias=False)
    (update_act): ReLU()
  )
  (pool3): TopKPooling(128, ratio=0.8, multiplier=1)
  (lin1): Linear(in_features=256, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=64, bias=True)
  (lin3): Linear(in_features=64, out_features=11, bias=True)
  (